## Feature Engineering & Justification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_preprocessing import * 
from feature_engineering import * 

c:\Users\olenk\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


#### Data Loading

In [25]:
stormEvents_2014 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2014_c20231116.csv")
stormEvents_2015 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2015_c20240716.csv")
stormEvents_2016 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2016_c20220719.csv")
stormEvents_2017 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2017_c20230317.csv")
stormEvents_2018 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2018_c20240716.csv")
stormEvents_2019 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2019_c20240117.csv")
stormEvents_2020 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2020_c20240620.csv")
stormEvents_2021 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2021_c20240716.csv")
stormEvents_2022 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2022_c20241121.csv")
stormEvents_2023 = pd.read_csv("../data/NOAA_StormEvents/StormEvents_details-ftp_v1.0_d2023_c20241216.csv")

stormEvents_dfs = [stormEvents_2014, stormEvents_2015, stormEvents_2016, stormEvents_2017, stormEvents_2018,
       stormEvents_2019, stormEvents_2020, stormEvents_2021, stormEvents_2022, stormEvents_2023]


In [42]:
eaglei_outages_2014 = pd.read_csv("../data/eaglei_data/eaglei_outages_2014.csv")
eaglei_outages_2015 = pd.read_csv("../data/eaglei_data/eaglei_outages_2015.csv")
eaglei_outages_2016 = pd.read_csv("../data/eaglei_data/eaglei_outages_2016.csv")
eaglei_outages_2017 = pd.read_csv("../data/eaglei_data/eaglei_outages_2017.csv")
eaglei_outages_2018 = pd.read_csv("../data/eaglei_data/eaglei_outages_2018.csv")
eaglei_outages_2019 = pd.read_csv("../data/eaglei_data/eaglei_outages_2019.csv")
eaglei_outages_2020 = pd.read_csv("../data/eaglei_data/eaglei_outages_2020.csv")
eaglei_outages_2021 = pd.read_csv("../data/eaglei_data/eaglei_outages_2021.csv")
eaglei_outages_2022 = pd.read_csv("../data/eaglei_data/eaglei_outages_2022.csv")
eaglei_outages_2023 = pd.read_csv("../data/eaglei_data/eaglei_outages_2023.csv")


outages_dfs = [eaglei_outages_2014, eaglei_outages_2015, eaglei_outages_2016, eaglei_outages_2017, eaglei_outages_2018,
               eaglei_outages_2019, eaglei_outages_2020, eaglei_outages_2021, eaglei_outages_2022, eaglei_outages_2023]

In [ ]:
weather_data = load_data('../data/weather_data/weather_data.csv')

__________________________________________________________________________________

### NOAA_StormEvents

In [38]:
def summarize_overall_missing(dfs, names=None):
    summaries = []
    
    for i, df in enumerate(dfs):
        dataset_name = names[i] if names else f"stormEvents_{2014+i}"
        
        missing_count = df['MAGNITUDE'].isna().sum()
        present_count = df['MAGNITUDE'].notna().sum()
        total_count = len(df)
        
        summaries.append({
            'DATASET': dataset_name,
            'MISSING_MAGNITUDE': missing_count,
            'PRESENT_MAGNITUDE': present_count,
            'MISSING_PERCENT': round((missing_count / total_count) * 100, 2)
        })
    
    return pd.DataFrame(summaries)

In [39]:
summarize_overall_missing(stormEvents_dfs)

,DATASET,MISSING_MAGNITUDE,PRESENT_MAGNITUDE,MISSING_PERCENT
0,stormEvents_2014,29969,29506,50.39
1,stormEvents_2015,29273,28634,50.55
2,stormEvents_2016,24661,31344,44.03
3,stormEvents_2017,23330,33699,40.91
4,stormEvents_2018,34063,28634,54.33
5,stormEvents_2019,32244,35617,47.51
6,stormEvents_2020,26418,34861,43.11
7,stormEvents_2021,31333,30056,51.04
8,stormEvents_2022,36148,33738,51.72
9,stormEvents_2023,35327,40269,46.73


One of the most important columns in the Storm Events dataset is **MAGNITUDE**, which allows us to assess the intensity of events. Unfortunately, for most events, this column is empty, with around 50% of the data missing in each dataset and around 60% of missing data in **MAGNITUDE_TYPE**. As a result, we will not be able to use it.

### Eaglei_Outages

POMYSŁ: może dołożymy procent ludzi bez prądu, w ten sposób będziemy miały miarę mówiącą o natężeniu braku dostawy. jakiego procenta ludzi możemy się spodziewać bez dostępu (to też w kontekście ułożenia historyjki będzie dobre)

Dobra byłoby miło gdyby nie fakt, że czasmia(a nawet często wyskakuje więcej niz 100%)

In [56]:
mcc = pd.read_csv("../data/eaglei_data/MCC.csv")
mcc["County_FIPS"] = mcc["County_FIPS"].astype(str)
dfs = [eaglei_outages_2020]

for i, df in enumerate(dfs):
    df["fips_code"] = df["fips_code"].astype(str)  
    df = df.merge(mcc, left_on="fips_code", right_on="County_FIPS", how="left") 
    df["percent_out"] = (df["customers_out"] / df["Customers"]) * 100
    dfs[i] = df.drop(columns=["County_FIPS"])



max_value = dfs[0]["percent_out"].max()
min_value = dfs[0]["percent_out"].min()

print(f"Max percent_out: {max_value}")
print(f"Min percent_out: {min_value}")

dfs[0][dfs[0]["percent_out"] > 100]

Max percent_out: 137300.0
Min percent_out: 2.6577366714505926e-05


,fips_code,county,state,customers_out,run_start_time,Customers,percent_out
148931,37095,Hyde,North Carolina,2608.0,2020-01-03 05:45:00,700.0,372.571429
149540,37095,Hyde,North Carolina,2608.0,2020-01-03 06:00:00,700.0,372.571429
150146,37095,Hyde,North Carolina,2608.0,2020-01-03 06:15:00,700.0,372.571429
150766,37095,Hyde,North Carolina,2608.0,2020-01-03 06:30:00,700.0,372.571429
151383,37095,Hyde,North Carolina,2608.0,2020-01-03 06:45:00,700.0,372.571429
...,...,...,...,...,...,...,...
25536588,37089,Henderson,North Carolina,43.0,2020-12-30 20:45:00,24.0,179.166667
25537339,37089,Henderson,North Carolina,43.0,2020-12-30 21:00:00,24.0,179.166667
25538083,37089,Henderson,North Carolina,43.0,2020-12-30 21:15:00,24.0,179.166667
25538823,37089,Henderson,North Carolina,43.0,2020-12-30 21:30:00,24.0,179.166667


### DATA CLEANING

Missing values in the latitude and longitude columns will be filled using the geopy library, which can convert a place (like a county) into its corresponding latitude and longitude.

In [ ]:
nom = ArcGIS()
def get_coordinates(county_name):
    try:
        location = nom.geocode(county_name)
        if location:
            return location.latitude, location.longitude
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Error geocoding {county_name}: {e}")
    return None, None

In [ ]:
for index, row in stormEvents_2014.iterrows():
    if pd.isna(row['BEGIN_LAT']) or pd.isna(row['BEGIN_LON']):
        lat, lon = get_coordinates(row['CZ_NAME'])
        if lat is not None and lon is not None:
            stormEvents_2014.at[index, 'BEGIN_LAT'] = lat
            stormEvents_2014.at[index, 'BEGIN_LON'] = lon
            stormEvents_2014.at[index, 'END_LAT'] = lat
            stormEvents_2014.at[index, 'END_LON'] = lon

In [ ]:
stormEvents_2014[['CZ_NAME', 'BEGIN_LAT', 'BEGIN_LON']]